15:24 09/11/2025

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from __future__ import annotations

import os
import sys
import time
from collections import deque
from pathlib import Path
from typing import Deque, Tuple, Optional

import cv2
import numpy as np
from ultralytics import YOLO

# ===========================
# CẤU HÌNH ĐƯỜNG DẪN & THAM SỐ
# ===========================

DATA_ROOT = Path("/home/gess/Documents/sub/TrainModel")
MODEL_DIR = DATA_ROOT / "models"
BEST_MODEL = MODEL_DIR / "best_drowsy.pt"  # đã train xong ở train_drowsy.py

# Class name phải trùng lúc train
CLS_OPEN = "open_eye"
CLS_CLOSED = "closed_eye"

# Tham số drowsy
HISTORY_LEN = 30        # số frame lưu history (khoảng 1 giây nếu 30fps)
DROWSY_THRESHOLD = 0.6  # tỉ lệ closed_eye / (open+closed) trong history
MIN_EYES_PER_FRAME = 1  # số mắt min để frame được tính

# Confidence ngưỡng để tính
CONF_THRESH = 0.5

# ===========================
# HÀM TIỆN ÍCH
# ===========================

def load_model(model_path: Path) -> YOLO:
    if not model_path.exists():
        print(f"[❌] Không tìm thấy model: {model_path}")
        sys.exit(1)
    print(f"[🚀] Load model: {model_path}")
    model = YOLO(str(model_path))
    return model


def decode_counts_from_result(
    result,
    cls_open: str = CLS_OPEN,
    cls_closed: str = CLS_CLOSED,
    conf_thresh: float = CONF_THRESH,
) -> Tuple[int, int]:
    """
    Đếm số open_eye và closed_eye trong 1 kết quả YOLO.

    Return:
        (num_open, num_closed)
    """
    names = result.names
    num_open = 0
    num_closed = 0

    if result.boxes is None or len(result.boxes) == 0:
        return 0, 0

    for b in result.boxes:
        cls_id = int(b.cls[0].item())
        conf = float(b.conf[0].item())
        if conf < conf_thresh:
            continue
        cls_name = names.get(cls_id, str(cls_id))
        if cls_name == cls_open:
            num_open += 1
        elif cls_name == cls_closed:
            num_closed += 1
    return num_open, num_closed


def update_drowsy_history(
    history: Deque[Tuple[int, int]],
    num_open: int,
    num_closed: int,
    maxlen: int = HISTORY_LEN,
) -> Tuple[float, int, int]:
    """
    Cập nhật history (deque) với (open, closed) của frame mới.
    Trả về:
        - closed_ratio (tỉ lệ closed / total)
        - total_open
        - total_closed
    """
    history.append((num_open, num_closed))
    if len(history) > maxlen:
        history.popleft()

    total_open = sum(o for o, _ in history)
    total_closed = sum(c for _, c in history)
    total = total_open + total_closed
    closed_ratio = (total_closed / total) if total > 0 else 0.0
    return closed_ratio, total_open, total_closed


def draw_info(
    frame: np.ndarray,
    num_open: int,
    num_closed: int,
    closed_ratio: float,
    is_drowsy: bool,
) -> np.ndarray:
    """
    Vẽ thông tin lên frame: số mắt mở/nhắm, tỉ lệ, trạng thái drowsy.
    """
    h, w = frame.shape[:2]
    overlay = frame.copy()

    # Bar mờ phía trên
    cv2.rectangle(overlay, (0, 0), (w, 80), (0, 0, 0), -1)
    alpha = 0.4
    frame = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)

    # Text info
    base_y = 25
    cv2.putText(
        frame,
        f"Open: {num_open}  Closed: {num_closed}",
        (10, base_y),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.7,
        (0, 255, 255),
        2,
        cv2.LINE_AA,
    )
    cv2.putText(
        frame,
        f"Closed ratio: {closed_ratio:.2f}",
        (10, base_y + 25),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.7,
        (255, 255, 0),
        2,
        cv2.LINE_AA,
    )

    status_text = "DROWSY!" if is_drowsy else "OK"
    color = (0, 0, 255) if is_drowsy else (0, 255, 0)
    cv2.putText(
        frame,
        f"Status: {status_text}",
        (10, base_y + 50),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.8,
        color,
        2,
        cv2.LINE_AA,
    )

    # Nếu drowsy thì vẽ khung cảnh báo
    if is_drowsy:
        cv2.rectangle(frame, (0, 80), (w - 1, h - 1), (0, 0, 255), 4)

    return frame


# ===========================
# WEBCAM REALTIME
# ===========================

def run_webcam(
    model: YOLO,
    cam_index: int = 0,
    history_len: int = HISTORY_LEN,
    drowsy_thresh: float = DROWSY_THRESHOLD,
):
    print(f"[🎥] Đang mở webcam {cam_index}...")
    cap = cv2.VideoCapture(cam_index)
    if not cap.isOpened():
        print("[❌] Không mở được webcam, thử index khác (0,1,2,...) hoặc kiểm tra quyền.")
        return

    history: Deque[Tuple[int, int]] = deque(maxlen=history_len)

    print("[INFO] Nhấn 'q' để thoát.")
    while True:
        ret, frame = cap.read()
        if not ret:
            print("[❌] Không đọc được frame từ webcam.")
            break

        # YOLO expect BGR → OK
        results = model.predict(
            source=frame,
            imgsz=768,
            conf=CONF_THRESH,
            verbose=False,
        )

        if len(results) == 0:
            num_open, num_closed = 0, 0
        else:
            num_open, num_closed = decode_counts_from_result(results[0])

        closed_ratio, total_open, total_closed = update_drowsy_history(
            history, num_open, num_closed, maxlen=history_len
        )

        is_drowsy = False
        total = total_open + total_closed
        if total >= MIN_EYES_PER_FRAME and closed_ratio >= drowsy_thresh:
            is_drowsy = True

        # Vẽ detection box của YOLO
        plotted = results[0].plot() if len(results) > 0 else frame.copy()

        # Vẽ info Drowsy
        plotted = draw_info(plotted, num_open, num_closed, closed_ratio, is_drowsy)

        cv2.imshow("Drowsy Detection - Webcam", plotted)
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()
    print("[✅] Đã thoát realtime.")


# ===========================
# PHÂN TÍCH 1 ẢNH
# ===========================

def analyze_image(model: YOLO, img_path: str | Path):
    img_path = Path(img_path)
    if not img_path.exists():
        print(f"[❌] Không tìm thấy ảnh: {img_path}")
        return

    img = cv2.imread(str(img_path))
    if img is None:
        print(f"[❌] Không đọc được ảnh: {img_path}")
        return

    results = model.predict(
        source=img,
        imgsz=768,
        conf=CONF_THRESH,
        verbose=False,
    )
    if len(results) == 0:
        print("[ℹ] Không có detection nào.")
        cv2.imshow("Drowsy - Image", img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        return

    r = results[0]
    num_open, num_closed = decode_counts_from_result(r)
    total = num_open + num_closed
    closed_ratio = (num_closed / total) if total > 0 else 0.0
    is_drowsy = closed_ratio >= DROWSY_THRESHOLD and total >= MIN_EYES_PER_FRAME

    print("===== PHÂN TÍCH ẢNH =====")
    print(f"Ảnh       : {img_path}")
    print(f"Open eyes : {num_open}")
    print(f"Closed    : {num_closed}")
    print(f"Closed ratio: {closed_ratio:.2f}")
    print(f"Trạng thái: {'DROWSY' if is_drowsy else 'OK'}")

    plotted = r.plot()
    plotted = draw_info(plotted, num_open, num_closed, closed_ratio, is_drowsy)
    cv2.imshow("Drowsy - Image", plotted)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


# ===========================
# PHÂN TÍCH FOLDER ẢNH
# ===========================

def analyze_folder(model: YOLO, folder: str | Path):
    folder = Path(folder)
    if not folder.exists():
        print(f"[❌] Không tìm thấy folder: {folder}")
        return

    exts = [".jpg", ".jpeg", ".png", ".bmp"]
    img_files = sorted(
        [p for p in folder.rglob("*") if p.suffix.lower() in exts]
    )
    if not img_files:
        print(f"[ℹ] Folder không có ảnh hợp lệ: {folder}")
        return

    print(f"[📂] Tìm thấy {len(img_files)} ảnh trong {folder}")
    for img_path in img_files:
        print(f"\n--- {img_path} ---")
        analyze_image(model, img_path)


# ===========================
# PHÂN TÍCH VIDEO
# ===========================

def analyze_video(
    model: YOLO,
    video_path: str | Path,
    history_len: int = HISTORY_LEN,
    drowsy_thresh: float = DROWSY_THRESHOLD,
):
    video_path = Path(video_path)
    if not video_path.exists():
        print(f"[❌] Không tìm thấy video: {video_path}")
        return

    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        print(f"[❌] Không mở được video: {video_path}")
        return

    history: Deque[Tuple[int, int]] = deque(maxlen=history_len)
    print("[INFO] Nhấn 'q' để thoát.")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = model.predict(
            source=frame,
            imgsz=768,
            conf=CONF_THRESH,
            verbose=False,
        )

        if len(results) == 0:
            num_open, num_closed = 0, 0
        else:
            num_open, num_closed = decode_counts_from_result(results[0])

        closed_ratio, total_open, total_closed = update_drowsy_history(
            history, num_open, num_closed, maxlen=history_len
        )
        total = total_open + total_closed
        is_drowsy = (total >= MIN_EYES_PER_FRAME) and (closed_ratio >= drowsy_thresh)

        plotted = results[0].plot() if len(results) > 0 else frame.copy()
        plotted = draw_info(plotted, num_open, num_closed, closed_ratio, is_drowsy)

        cv2.imshow("Drowsy Detection - Video", plotted)
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()
    print("[✅] Đã xong video.")


# ===========================
# MAIN: MENU ĐƠN GIẢN (KHÔNG CẦN CLI)
# ===========================

def main():
    """
    Chạy trực tiếp file (VS Code Run) → hiện menu:
      1. Webcam
      2. Ảnh
      3. Folder ảnh
      4. Video
    """
    model = load_model(BEST_MODEL)

    print("\n====================")
    print("  DROWSY DETECTION  ")
    print("====================")
    print("1) Webcam realtime")
    print("2) Phân tích 1 ảnh")
    print("3) Phân tích 1 folder ảnh")
    print("4) Phân tích 1 video")
    print("0) Thoát")
    choice = input("Chọn mode (0-4): ").strip()

    if choice == "1":
        idx = input("Nhập camera index (mặc định 0): ").strip()
        cam_idx = int(idx) if idx else 0
        run_webcam(model, cam_index=cam_idx)
    elif choice == "2":
        path = input("Nhập đường dẫn ảnh: ").strip()
        analyze_image(model, path)
    elif choice == "3":
        folder = input("Nhập folder ảnh: ").strip()
        analyze_folder(model, folder)
    elif choice == "4":
        path = input("Nhập đường dẫn video: ").strip()
        analyze_video(model, path)
    else:
        print("Thoát.")


if __name__ == "__main__":
    main()
